# Foyer Tutorial 01: SMARTS and Overrides

This tutorial focuses on the basic usage of SMARTS to define chemical context and the use of overrides to set rule precedence, and demonstrates how to use a force field file to atomtype a system.

As shown in the prior tutorial, atom-type usage (i.e., chemical context) is defined via a SMARTS string contained in the `def` tag.  Below is the AtomTypes subsection of the XML file for a forcefield that describes alkanes using the OPLS all-atom force field (as detailed in [Jorgensen, et al.](http://dx.doi.org/10.1021/ja9621760)). 

In [2]:
! sed -n 2,18p OPLSaa_alkanes.xml

    <AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760" overrides="opls_136"/>

        <Type name="opls_136" def="[C;X4](H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760" overrides="opls_135,opls_136"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>


### Defining SMARTS
Focusing first on atom type `opls_140`, the SMARTS string, `def="H[C;X4]"`, states that this atom-type applies when:
- The element is hydrogen, i.e., `H`
- When that hydrogen is connected to a carbon that has 4 neighbors, i.e., `[C;X4]`

Similarly, for atom type `opls_138`, the SMARTS string, `def="[C;X4](H)(H)(H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- 4 of those neighbors are hydrogens, i.e., `(H)(H)(H)H`

For atom type `opls_136`, the SMARTS string, `def="[C;X4](H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 2 of those neighbors are hydrogens, i.e., `(H)H`

For atom type `opls_135`, the SMARTS string, `def="[C;X4](H)(H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 3 of those neighbors are hydrogens, i.e., `(H)(H)H`

### Atomtyping using SMARTS
Let us now consider using these rules to atom-type the carbon in methane (i.e., CH4).

- `opls_138` would obviously evaluate to `True`, as it is defined for a carbon, with 4 hydrogen neighbors. 
- `opls_135` and `opls_136` would also evaulate to `True`.  In the case of opls_135, our definition only states that at least 3 of the neighbors are hydrogen, and have not made any specific claims about the identity of the 4th neighbor; similarly, for opls_136, we have only stated that 2 neighbors must be hydrogen and not specified the identity of the other 2 neighbors. 

This is an important feature of Foyer to take note of.  Foyer will evaluate all the rules in a forcefield file, rather than just stopping at the first rule that evaluates to `True`. This allows rules to be defined in any order.  Furthermore, Foyer iterates over the rules, which allows recursive definitions of usage, i.e., referring to specific atom-types in the SMARTS string. 

### Overrides
We will discuss two ways to address this. One approach is to employ `overrides`.  `overrides` provide a means to dictate rule precedence (i.e., which rules are more specific than others).  In the force field file above, `opls_138` has defined: `overrides="opls_135,opls_136"`.  That is, if `opls_138` evaluates to `True`, then it takes precedence over `opls_135` and `opls_136`, even if they evaluate to `True`. 

Similarly, if opls_136 evaluates to `True`, it `overrides="opls_136"`, thus taking precedence. 

`overrides` are especially useful if the chemical context of two different atom-types are effectively the same.  E.g., the terminal methyl group in an alkane has the same first neighbor environment as the methyl group in toluene, however the parameters (and thus atom-type) are different. Thus `overrides` can be used to state that if the more specific toluene rule evaluates to `True` it should take precedences over the more general alkane rule (as shown below):

<img src="ch3-toluene.png" alt="Drawing" style="width: 700px;"/>

### Better SMARTS definitions
In many cases, `overrides` can be avoided by simply providing more specific SMARTS strings.  For example, the rules for `opls_135`, `opls_136`, and `opls_138` above can be made more specific by stating the identify of the other neighbors besides carbon and thus eliminate the need for `overrides`, as shown below. 

In [4]:
! sed -n 2,18p OPLSaa_alkanes2.xml

    <AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)(H)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760"/>

        <Type name="opls_136" def="[C;X4](H)(H)(C)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>


Note that SMARTS can be used to define more than just the immediate local pattern. Recall the definition for `opls_140`, `def='H[C;X4]'`. This definition states not only that the element is hydrogen and is bonded to a carbon, but that the carbon it is bonded to has 4 total bonds. If necessary, the identify of those bonded neighbors could  be specified. 

Additionally, we note that since this force field only has a single hydrogen atom-type, the definition could actually be made less specific, i.e., `def='H'`, and still produce the correct output.  

Thus, it is important to keep in mind that there are multiple valid SMARTS definitions that can be provided for a given atom-type; the specificity of the SMARTS definition and whether or not `overrides` are necessary will depend on the chemical context of the parameters themselves, the number of atom-types included in a forcefield, the intended usage of the forcefield, and the personal preferences of the individual(s) defining the forcefield.   

### Applying a force field to atomtype a system
With a force field file defined, we will apply it to an alkane system.  Here, we will use the routines previously discussed in the mBuild tutorial to construct a hexane, then perform atom-typing as part of saving our configuration to a desired file format.

Note, the following command sequence may provide a long list of Deprecation Warnings (which will be suppressed in the next mBuild release).

In [13]:
%matplotlib notebook
from visualize import visualize
import mbuild as mb

from mbuild.examples import Alkane

hexane = Alkane(6)

hexane.save('hexane.top', forcefield_files='OPLSaa_alkanes.xml', overwrite=True)
hexane.save('hexane.lammps', forcefield_files='OPLSaa_alkanes.xml', overwrite=True)

/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\#'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\['
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\]'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\('
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\)'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\,'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/

We can easily view the output files. 

In [14]:
# view the gromacs formatted file
!cat hexane.top

;
;   File hexane.top  was generated
;   By user: cri (503)
;   On host: IacBook-Pro-Retina.local
;   At date: Wed. November  3 21:05:47 2017
;
;   This is a standalone topology file
;
;   Created by:
;   ParmEd:       __main__.py, VERSION 2.7.3
;   Executable:   __main__.py
;   Library dir:  /usr/local/gromacs/share/gromacs/top
;   Command line:
;     /Users/cri/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py -f /Users/cri/Library/Jupyter/runtime/kernel-ce609562-7cff-4a5f-ab9f-84ce8586447a.json
;

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               2               no              1           0.5

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
opls_136        6   12.01078    0.000000  A          0.35      0.276144
opls_140        1    1.00795    0.000000  A          0.25       0.12552
opls_135        6   12.01078    0.000000  A          0.35      0.276144


[ moleculetype ]
; Name            nrexcl
RES    

In [15]:
# view the lammps formatted file
!cat hexane.lammps

hexane.lammps - created by mBuild

20 atoms
19 bonds
36 angles
45 dihedrals

3 atom types
2 bond types
3 angle types
2 dihedral types

0.000000 7.200000 xlo xhi
0.000000 13.538000 ylo yhi
0.000000 6.646000 zlo zhi

Masses

2	12.010780	# opls_136
3	1.007947	# opls_140
1	12.010780	# opls_135

Pair Coeffs # lj

2	0.06600	3.50000
3	0.03000	2.50000
1	0.06600	3.50000

Bond Coeffs # harmonic

1	340.0	1.09
2	268.0	1.529

Angle Coeffs # harmonic

1	37.5	110.70000
2	58.35	112.70000
3	33.0	107.80000

Dihedral Coeffs # opls

1	1.30000	-0.05000	0.20000	-0.00000
2	-0.00000	0.00000	0.30000	-0.00000

Atoms

1	0	2	-0.120000	0.000000	-5.600000	-0.000000
2	0	3	0.060000	-1.100000	-5.600000	-0.000000
3	0	3	0.060000	1.100000	-5.600000	-0.000000
4	0	2	-0.120000	0.000000	-4.200000	-0.000000
5	0	3	0.060000	-1.100000	-4.200000	-0.000000
6	0	3	0.060000	1.100000	-4.200000	-0.000000
7	0	2	-0.120000	0.000000	-2.800000	-0.000000
8	0	3	0.060000	-1.100000	-2.800000	-0.000000
9	0	3	0.060000	1.100000	-2.800000	-0.000000

Forcefield files allow a DOI for the original parameter source to be defined, thus enabling Foyer to output a bibtex formatted references file, that clearly states in the notes which atom-types are associated with a given reference. 

In [16]:
hexane.save('hexane.top', forcefield_files='OPLSaa_alkanes.xml', references_file='hexane.bib', overwrite=True)

/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\#'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\['
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\]'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\('
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\)'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/plyplus/plyplus.py:758: DeprecationWarning: invalid escape sequence '\,'
  return codecs.getdecoder('unicode_escape')(token_value)[0]
/Users/cri/Projects/plyplus/

In [17]:
cat hexane.bib

@article{Jorgensen_1996,
	doi = {10.1021/ja9621760},
	url = {https://doi.org/10.1021%2Fja9621760},
	year = 1996,
	month = {jan},
	publisher = {American Chemical Society ({ACS})},
	volume = {118},
	number = {45},
	pages = {11225--11236},
	author = {William L. Jorgensen and David S. Maxwell and Julian Tirado-Rives},
	title = {Development and Testing of the {OPLS} All-Atom Force Field on Conformational Energetics and Properties of Organic Liquids},
	journal = {Journal of the American Chemical Society},
	note = {Parameters for atom types: opls_140, opls_136, opls_135}
}
